<a href="https://colab.research.google.com/github/AnupamaRajkumar/AppliedDeepLearning/blob/master/Keypoint_detection_on_Cats_Datset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1/B
**Disclaimer: Only for ADL/AML students!**

### General information
**You have to solve all tasks to pass!** 

Grade is calculated by the day of the last submission, but you will only get it after you've succesfully presented it. 

**Deadlines and grades:** 
  * 2020.09.20 - 2020.10.27 ==> 5
  * 2020.10.28 - 2020.11.03 ==> 4
  * 2020.10.04 - 2020.11.10 ==> 3
  * 2020.11.11 - 2020.11.17 ==> 2
  * 2020.11.18 or later ==> 1 

You can **use only these** 3rd party **packages:** `cv2, keras, matplotlib, numpy, pandas, sklearn, skimage, tensorflow`.

### Description
In this assignment you have to build and train a cat eyes and nose keypoint detector model using tf.keras. We will use an autoencoder like architecture, which first encodes the data, then decodes it to its original size. To implement such kind of models, you should take a look at the following classes and methods: `Funcitonal API, MaxPooling2D, Conv2DTranspose`.

### Use GPU
Runtime -> Change runtime type

At Hardware accelerator select  GPU then save it.  

### Useful shortcuts
* Run selected cell: *Ctrl + Enter*
* Insert cell below: *Ctrl + M B*
* Insert cell above: *Ctrl + M A*
* Convert to text: *Ctrl + M M*
* Split at cursor: *Ctrl + M -*
* Autocomplete: *Ctrl + Space* or *Tab*
* Move selected cells up: *Ctrl + M J*
* Move selected cells down: *Ctrl + M K*
* Delete selected cells: *Ctrl + M D*


## Prepare dataset

* Download the Cats dataset. We will only use a subset of the original dataset, the CAT_00 folder. Here you can find more information about the dataset: https://www.kaggle.com/crawford/cat-dataset
* Preprocess the data. You can find some help here: https://github.com/kairess/cat_hipsterizer/blob/master/preprocess.py
  * Following the steps in the link above, read and resize the images to be 128x128.
  * Keep only the left eye, right eye and mouth coordinates.
  * Create a keypoint heatmap from the keypoints. A 128x128x3 channel image, where the first channel corresponds to left eye heatmap, the sencond channel corresponds to the right eye heatmap and the third channel corresponds to the mouth heatmap. To do this:
    1. At each keypoint, draw a circle with its corresponding color. For this, use the following method: `cv2.circle(<heatmap>, center=<keypoint_coord>, radius=2, color=<keypoint_color>, thickness=2)`
    2. Then smooth the heatmap with a 5x5 Gauss filter: `<heatmap> = cv2.GaussianBlur(<heatmap>, (5,5), 0)` 
  * Then crop each image and heatmap:
    1. Define the bounding box, select the min and max keypoint coordinates: `x1, y1, x2, y2`.
    2. Add a 20x20 border around it: `x1, y1, x2, y2 = x1-20, y1-20, x2+20, y2+20`.
    3. Then crop the image and the heatmap using the extended bounding box.  
  * Finally, resize the images and the heatmaps to be 64x64.
* Split the datasets into train-val-test sets (ratio: 60-20-20), without shuffling.
* Print the size of each set and plot 5 training images and their corresponding masks.
* Normalize the datasets. All value should be between 0.0 and 1.0. *Note: you don't have to use standardization, you can just divide them by 255.*

In [ ]:
# Download from Drive
!if ! [ -f CAT_00.zip ]; then curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1wGwNi8t-UKAKs-LQL3dG-D8dzGVPHv2w" > /dev/null; curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1wGwNi8t-UKAKs-LQL3dG-D8dzGVPHv2w" -o CAT_00.zip; fi

# Check if the file size is correct (~402MB)
!if (( $(stat -c%s CAT_00.zip) < 421896648 )); then rm -rfd CAT_00.zip; fi

# If not, download it from NIPG12
!wget -nc -O CAT_00.zip http://nipg1.inf.elte.hu:8000/CAT_00.zip

In [ ]:
!unzip CAT_00.zip

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas
import skimage
import sklearn

## Data augmentation
  * Augment the training set using `ImageDataGenerator`. The parameters should be the following: `featurewise_center=False, featurewise_std_normalization=False, width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.2`.
  * When creating the generator(s), use shuffling with a seed value of 1 and batch size of 32.
  * To validate that the augmentation is working, plot 5 original images with their corresponding transformed (augmented) images and masks.

**Keep in mind:** To augment the inputs and targets the same way, you should create 2 separate generator, then you can zip them together. 


## Define the model
Define the following architecture in tf.keras:
```
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_1 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
bottleneck_1 (Conv2D)        (None, 32, 32, 160)       5243040   
_________________________________________________________________
bottleneck_2 (Conv2D)        (None, 32, 32, 160)       25760     
_________________________________________________________________
upsample_1 (Conv2DTranspose) (None, 64, 64, 3)         1920      
=================================================================
Total params: 5,530,880
Trainable params: 5,530,880
Non-trainable params: 0
_________________________________________________________________
```
* Use relu and `padding='same'` for each layer.
* Use a 2x2 `Conv2DTranspose` layer without bias to upsample the result. 
* `block1_conv1`, `block1_conv2`, `block2_conv1` and `block2_conv2` are 3x3 convolutions.
* `bottleneck_1` is a 16x16 and `bottleneck_2` is a 1x1 convolution.
* For optimizer use RMSProp, and MSE as loss function.


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *

In [ ]:
def fcnet(input_size=(64, 64, 3)):
  pass

model = fcnet(input_size=(64, 64, 3))
model.summary()

## Training and evaluation 
  * Train the model for 30 epochs without early stopping.
  * Plot the training curve (train/validation loss).
  * Evaluate the trained model on the test set.
  * Plot some (5) predictions with their corresponding GT heatmap and input. You should mark the location of each keypoint on the image. *Note: it might be worth to take a look at this answer: https://stackoverflow.com/a/17386204*